# AIS Port Classification notebook

This notebook is a walktrough of the AIS Marinetime Project by group 135. In this project, we have focused on Port Classification given time series data. 

The notebook consits of the following contents:
- [Introduction](#introduction-to-methods)
- [Setup](#setup) 
- [Data](#Data-exploration) 
    - [Data source & preparation](#data-sources--ingestion) 
    - [Descriptive statistics & Data Visualization](#descriptive-statistics) 
    - [Feature Engineering](#feature-engineering)

# Introduction 

In this section, we will introduce the goal and methods in this project. We will present the different models and techniques used for port classification using AIS data. 

## The Goal of the Project 
The goal of this project is to build a model that can accurately classify ports based on AIS data. 
To achive this, we will have used 3 deep learning models and one simple machine learning model as a baseline. The models used are:  
1. LSTM - Long Short-Term Memory network 
2. Transformer
3. LSTM-Transformer Hybrid Model
4. "TO DO " (baseline) 


For each model, the output is from a feed forward neural network with can be toughts of as logits. 

We use the cross-entropy loss function to train the models, as it is well suited for multi-class classification problems. 



# Setup

Before running this notebook, make sure to install the required dependencies listed in the `pyproject.toml` file and have access to the dataset used in this project.

This project uses `uv` as the Python package manager. All dependencies are defined in `pyproject.toml`.

Run the following commands to set up your environment:

```bash
# Install all dependencies from pyproject.toml
uv sync

# Install the project in editable mode (to import from src/)
uv pip install -e .
```

The cleaned dataset used in this project can be found here: [https://drive.google.com/drive/folders/1o17rqpOwHRdrgkoRS8rd5g62xi9nsTuk?hl=da] 

The folder is called "processed_data" and should be zipped and placed in the data directory of this project.

Otherwise, you can down the data in the next cell. 



In [77]:
import matplotlib.pyplot as plt
import pandas as pd 
from Dataloader import Dataloader 
import numpy as np 
import torch  
from RNN import clean_data, setup_and_train
from utils import MaritimeDataset, plot_trajectory_on_map, download
import os
import zipfile 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import mlflow 
import gc

In [81]:
config = {

    "TRAIN_OWN_MODEL": True, 


    "DOWNLOAD_DATA_FROM_SOURCE": False, #Set to True if data needs to be downloaded from source
    "Experiment_Name": "Test Notebook",
    #"Experiment_Name": "Classifier-Experiment - Full Run 1: 25.Nov.2025", 


    ##Paths for saving and loading models and data
    "LOAD_CLEANED_DATA_PATH": "data/cleaned_data.parquet", 
    "DATA_PATH": "data/processed_data", 
}


#By chaning the hyperparams, it should be noted that no Pretrained models exist
hyperparams = {
        "general": {
        "batch_size": 128,
        "num_epochs": 50,
        "learning_rate": 0.0001,
        },

        #Model specific hyperparameters
        "LSTM": {
        "lstm_hidden_size": 64,
        "lstm_num_layers": 5,
        "dropout": 0.1,
        "batch_first": True
        }, 

        "LSTM_Transformer": {
        "hidden_size": 256,
        "num_lstm_layers": 5,
        "num_heads": 8,
        "num_transformer_layers": 6,
        "dropout": 0.1,
        "batch_first": True
        }
    }


#Setting seed for reproducibility 
torch.manual_seed(42)
np.random.seed(42)
torch.cuda.manual_seed_all(42)  



# Data

Downloading data from source:

In [ ]:
if (config["DOWNLOAD_DATA_FROM_SOURCE"]):    
    data_links = {"Jan": "http://aisdata.ais.dk/2023/aisdk-2023-01.zip",
                  "Feb": "http://aisdata.ais.dk/2023/aisdk-2023-02.zip",
                  "Mar": "http://aisdata.ais.dk/2023/aisdk-2023-03.zip",
                  "Apr": "http://aisdata.ais.dk/2023/aisdk-2023-04.zip",
                  "May": "http://aisdata.ais.dk/2023/aisdk-2023-05.zip",
                  "Jun": "http://aisdata.ais.dk/2023/aisdk-2023-06.zip",
                  "Jul": "http://aisdata.ais.dk/2023/aisdk-2023-07.zip",
                  "Aug": "http://aisdata.ais.dk/2023/aisdk-2023-08.zip",
                  "Sep": "http://aisdata.ais.dk/2023/aisdk-2023-09.zip",
                  "Oct": "http://aisdata.ais.dk/2023/aisdk-2023-10.zip",
                  "Nov": "http://aisdata.ais.dk/2023/aisdk-2023-11.zip",
                  "Dec": "http://aisdata.ais.dk/2023/aisdk-2023-12.zip"
                 }

    data_dir = "data/unprocessed"
    end_dir = "data/processed"



    os.makedirs(data_dir, exist_ok=True)
    os.makedirs(end_dir, exist_ok=True)

    for month, link in data_links.items():
        filename = link.split("/")[-1]
        filepath = os.path.join(data_dir, filename)

        # Extract month number from filename
        month_num = filename.split("-")[2].replace(".zip", "")

        # Check if all parquet files for this month already exist
        # Quick check: if folder has files matching this month pattern
        existing_parquets = [f for f in os.listdir(end_dir) if f.startswith(f"aisdk-2023-{month_num}-") and f.endswith('.parquet')]

        # A month should have 28-31 parquet files
        if len(existing_parquets) >= 28:
            print(f"⏭️  Skipping month {month}: {len(existing_parquets)} parquet files already exist")
            continue

        # Download if needed
        if os.path.exists(filepath):
            print(f"⏭️  Skipping download for month {month}: {filepath} (already exists)")
        else:
            print(f"Downloading data for {month}...")
            download(link, filepath)
            print(f"Downloaded data for {month}")

        # Open ZIP once and process all CSV files
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            csv_files = [f for f in zip_ref.namelist() if f.endswith('.csv')]
            print(f"Found {len(csv_files)} CSV files in {filename}")

            for csv_filename in csv_files:
                output_filename = csv_filename.replace('.csv', '.parquet')
                output_path = os.path.join(end_dir, output_filename)

                if os.path.exists(output_path):
                    print(f"⏭️  Skipping {output_filename} (already exists)")
                    continue

                print(f"📝 Processing: {csv_filename}")

                data = Dataloader(
                    file_path="",
                    out_path=output_path,
                    zip_path=filepath,
                    csv_internal_path=csv_filename
                )
                data.clean_data()

        print(f"✅ Processed {month}")

        # Remove ZIP after processing 
        os.remove(filepath)
        print(f"🗑️  Removed {filename}")

    print("🎉 All data processed")


## Data source & Preparation 




All the data is being prerocessed and cleaned in the prepare_data.ipynb notebook.

We do the following steps in the data preparation:
1. Download 2023 raw data from [AIS data source](http://aisdata.ais.dk/)
2. We only keep the column that we deem are relevant for our project i.e 
    - MMSI
    - SOG 
    - COG
    - Longitude
    - Latitude
    - Timestamp 
    - Ship type 
    - Type of mobile  
    - Length 
    - Width 
    - Destination 

3. We filter the following data points: 
    - data points not in the Danish waters. 
    - data points which are classfied as "Ship type A" and then remove all other ship types. (Type of mobile is redundant after this step and removed).  
    - 1 <= SOG <= 50 knots 
    - Sequences below 250 data points are removed. 
    - data with non-valid port codes are removed (/data_port_locodes.csv are used a source of truth)
    - Filter anomalities where the contains jump in data points that are not realistic (based on SOG and distance between points). 


4. We do the following transformations:
    - Label the data points for each date into segments
    - Datapoints are concated based on segments such that each sequence of a ship can stretch over multiple days. 
    - downsampling such that there is at least 10 seconds between each data point (if two points are closer than 10 seconds, we remove the first point). 




In [82]:
if (config["DOWNLOAD_DATA_FROM_SOURCE"]):
    path = config["DATA_PATH"]
    dataloader = Dataloader(out_path=path)
    df = dataloader.load_data()  # load all files in the processed_data folder
    #Same cleaning as in RNN 
    df_clean = clean_data(df) 
    df_clean.to_parquet(config["LOAD_CLEANED_DATA_PATH"], index=False)  

else: 
    df_clean = pd.read_parquet(config["LOAD_CLEANED_DATA_PATH"]) 
    

In [ ]:
df_clean

## Descriptive statistics & Data Visualization

First we check how the distribution of the different ports are in the cleaned dataset. 

In [ ]:
ports = df_clean['Port'] 
ports_norm = ports.value_counts(normalize=True)  # Count occurrences of each port

# Plot top 10 ports as bar chart
top_10 = ports_norm.head(10)
plt.figure(figsize=(12, 6))
top_10.plot(kind='bar')
plt.xlabel('Port', fontsize=12)
plt.ylabel('Proportion', fontsize=12)
plt.title('Top 10 Most Frequent Ports (Normalized)', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

Because we see that there is a lot of inbalance between the groups, we will need to balance this out succh that our model doesnt just predict the "no_port" class. 

To solve this problem, we will introduce class weights. We use skilearn's `compute_class_weight` function to calculate the class weights based on the frequency of each class in the training data. 

In [ ]:
#plot_trajectory_on_map(df_clean, percentage_of_vessels=0.1)

To check if seasonality plays a role in the data, we plot the weekly observations for each port type

In [ ]:
def plot_weekly_observations(df, timestamp_col="Timestamp"):
    df_copy = df.copy()
    df_copy[timestamp_col] = pd.to_datetime(df_copy[timestamp_col])
    

    df_copy['YearWeek'] = df_copy[timestamp_col].dt.to_period('W')
    weekly_counts = df_copy['YearWeek'].value_counts().sort_index()
    
    weekly_counts.index = weekly_counts.index.to_timestamp()
    
    # Plot
    plt.figure(figsize=(14, 6))
    plt.plot(weekly_counts.index, weekly_counts.values, marker='o', linewidth=1.5, markersize=4)
    plt.xlabel('Week', fontsize=12)
    plt.ylabel('Number of Observations', fontsize=12)
    plt.title('Weekly AIS Observations Over Time', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"Total weeks: {len(weekly_counts)}")
    print(f"Total observations: {weekly_counts.sum():,}")
    print(f"Average observations per week: {weekly_counts.mean():.0f}")
    print(f"Min observations in a week: {weekly_counts.min():,}")
    print(f"Max observations in a week: {weekly_counts.max():,}")
    
    return weekly_counts


weekly_obs = plot_weekly_observations(df_clean) 

It looks like there is some seasonality in the data, with peaks in the summer months and troughs in the winter months. To invistiage this further and if it have a impact on our ports, we plot the ports observations over time. 

In [ ]:
for port_type in top_10.index:
    print(f"Plotting weekly observations for ship type: {port_type}") 
    df_type = df_clean[df_clean['Port'] == port_type]
    plot_weekly_observations(df_type) 

Its now very clear that there is seasonality in the data for each port.  

To adress this, we wil not use absolute timestamps as input features to our models. Instead, we will use relative time features. This way, the model can learn to generalize across different time periods without being affected by seasonality. 

We now look at the distribution of the ship types in the cleaned dataset.

In [ ]:
ship_types = df_clean['Ship type'] 
ship_types_norm = ship_types.value_counts(normalize=True)  # Count occurrences of each ship type

# Plot top 10 ship types as bar chart
top_10 = ship_types_norm.head(10)
plt.figure(figsize=(12, 6))
top_10.plot(kind='bar')
plt.xlabel('Ship Type', fontsize=12)
plt.ylabel('Proportion', fontsize=12)
plt.title('Top 10 Most Frequent Ship Types (Normalized)', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

We see that the majority of the ships are cargo ships, followed by tankers and passenger ships. It will be considered in future work to feature engineer the data based on ship type to see if this improves the model performance. But for this project, we will not consider ship type as a feature.

## Feature Engineering

To make sure we consider features that can help the model perform better, we do a PCA analysis to see which features are the most important. 

We have already done a few feature engineering steps in the data preparation notebook, such as: 
- Finding cos & sin of COG to represent direction better.

In [ ]:
train_df, test_df = dataloader.train_test_split(df = df_clean, prediction_horizon_hours=2.0)
train_df, val_df = dataloader.train_test_split(df = train_df, prediction_horizon_hours=0, test_size=0.2) # should not remove further hours for validation as 2 hours are already removed

train_dataset = MaritimeDataset(train_df)
print("Feature Engineering: New columns in input data are")
train_dataset.feature_cols

We now do a PCA analysis on our train set to wich features are the most important. 

In [ ]:

tensor_list = train_dataset.sequences

k = len(train_dataset.feature_cols) 

# Concatenate all tensors
all_data = torch.cat(tensor_list, dim=0).numpy()  # Shape: (total_timesteps, n_features)

# Fit PCA once on all data
scaler = StandardScaler()
pca = PCA(n_components=k)

all_data_scaled = scaler.fit_transform(all_data)

pca.fit(all_data_scaled)

# Explained variance ratio for each component
explained_var = pca.explained_variance_ratio_

plt.figure(figsize=(10, 6))
plt.bar(range(len(explained_var)), explained_var)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Variance Explained by Each Component')
plt.show()

# Cumulative explained variance
cumulative_var = np.cumsum(explained_var)
plt.figure(figsize=(10, 6))
plt.plot(cumulative_var, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Variance Explained')
plt.grid(True)
plt.show()


We see that no feature explain under 10% of the variance, which means that all features are important for the model and should be kept. The high variance per feature also indicates that there is low multicollinearity between the features, which is a good sign for the model performance. 

To check for multicollinearity between the features, we also plot a correlation matrix.

In [ ]:

corr_matrix = pd.DataFrame(all_data_scaled, columns=train_dataset.feature_cols).corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=0.5)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()


threshold = 0.8
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            high_corr_pairs.append((i, j, corr_matrix.iloc[i, j]))

print("Highly correlated feature pairs:")
for i, j, corr in high_corr_pairs:
    print(f"Feature {i} & Feature {j}: {corr:.3f}")

We see that there a no highly correlated features (correlation > 0.8), which indicates low multicollinearity between the features. Therefore, the data for the model is clean and ready for training. 

We now train (or load pretrained) models. 

For the full training details, see the **RNN.py** file.

In [ ]:
models_to_train = ["LSTM", "LSTM_Transformer"] 

client = mlflow.MlflowClient() 

experiment = client.get_experiment_by_name(config["Experiment_Name"]) 

if (experiment is None and config["TRAIN_OWN_MODEL"]):
    experiment_id = client.create_experiment(config["Experiment_Name"])

elif (experiment is None and not config["TRAIN_OWN_MODEL"]):
    raise ValueError(f"Experiment '{config['Experiment_Name']}' does not exist. Cannot load pretrained model.")  

else:
    experiment_id = experiment.experiment_id 


Training own model

In [ ]:
if (config["TRAIN_OWN_MODEL"]):

    test_len = len(test_df)
    val_len = len(val_df)
    train_len = len(train_df)
    total_len = test_len + val_len + train_len

    #setting hyperparams
    params = { "test_size": test_len / total_len,
        "val_size": val_len / total_len,
        "train_split": train_len / total_len,
        }

    hyperparams["general"] = {**hyperparams["general"], **params}

    mlflow.set_experiment(config["Experiment_Name"])

    for model_name in models_to_train:
        print(f"Training model: {model_name}")
        with mlflow.start_run(run_name=f"{model_name}-Run"):
            #Logging params for later analysis
            mlflow.log_params(hyperparams['general'])
            mlflow.log_params(hyperparams[model_name])
            mlflow.log_param("model_type", model_name) 

            
            trained_model = setup_and_train(train_df=train_df, 
                                            val_df=val_df, 
                                            test_df=test_df, 
                                            hyperparams=hyperparams, 
                                            model=model_name)

        #clean up 
        del trained_model 
        gc.collect() 
        if (torch.cuda.is_available()):
            torch.cuda.empty_cache()

Training model: LSTM
The 'no_port' ID is successfully set to: 99
The 'no_port' ID is successfully set to: 99
Grouping data into sequences...
Grouping data into sequences...
Created 6663 sequences.
Class weights calculated: tensor([6.6630e+01, 8.3287e+00, 5.2055e-01, 1.3326e+01, 8.3287e+00, 6.6630e+01,
        3.5068e+00, 3.3315e+01, 1.6657e+01, 4.9356e-01, 4.1644e+00, 4.4420e+00,
        1.6657e+01, 6.6630e+01, 3.4885e-01, 3.3315e+01, 1.1105e+01, 3.0286e+00,
        2.2210e+01, 9.5186e+00, 2.2210e+00, 1.4485e+00, 8.7671e-01, 6.0573e+00,
        9.5186e+00, 1.6657e+01, 6.6630e+01, 4.8635e-01, 8.1256e-01, 7.4033e+00,
        3.3315e+01, 2.7762e+00, 2.0822e+00, 1.6657e+01, 1.0576e+00, 6.0573e-01,
        9.5186e+00, 1.6657e+01, 1.9597e+00, 8.3287e+00, 2.2210e+01, 5.5525e+00,
        3.3315e+01, 6.6630e+01, 2.3796e+00, 3.5068e+00, 3.7017e+00, 1.0747e+00,
        1.2813e+00, 2.4678e+00, 3.9194e+00, 9.5186e+00, 6.6630e+00, 9.5186e+00,
        3.3315e+00, 2.2210e+01, 1.6657e+01, 3.3315e+01, 3

TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

In [65]:
# Get all runs from the experiment
all_runs = client.search_runs(
    experiment_ids=[experiment_id],
    order_by=["start_time DESC"]
)

print(f"Total runs found: {len(all_runs)}\n")

# Go through runs and check which have non-empty metrics
for i, run in enumerate(all_runs):
    run_id = run.info.run_id
    metrics = run.data.metrics
    
    if metrics:  # Check if metrics dict is not empty
        print(f"Run {i+1}: {run_id}")
        print(f"  Status: {run.info.status}")
        print(f"  Metrics: {metrics}")
        print()
    else:
        print(f"Run {i+1}: {run_id} - No metrics recorded")
        print()

Total runs found: 3

Run 1: 4f4752e38e1048caa87255516d83fc38 - No metrics recorded

Run 2: ad0faa9dfcbc44b4ba0d90d84037921b
  Status: FINISHED
  Metrics: {'train_loss': 1.9527696041833786, 'val_accuracy': 53.164556962025316}

Run 3: c1e115e23eb94b438a8a32d137843dc2
  Status: FINISHED
  Metrics: {'train_loss': 1.9792297874178206, 'val_accuracy': 50.33152501506932}



------------------- RANDOM EDITS BELOW -------------------

In [ ]:
import matplotlib.pyplot as plt

def plot_daily_observations(df, timestamp_col="Timestamp"):
    """
    Groups dataframe by week, counts observations, and plots the time series.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing the data
    timestamp_col : str
        Name of the timestamp column (default: "Timestamp")
    
    Returns:
    --------
    pd.Series
        Weekly observation counts indexed by week start date
    """
    # Convert to datetime if not already
    df_copy = df.copy()
    df_copy[timestamp_col] = pd.to_datetime(df_copy[timestamp_col])
    
    # Extract year-week and count observations per week
    df_copy['YearWeek'] = df_copy[timestamp_col].dt.to_period('W')
    weekly_counts = df_copy['YearWeek'].value_counts().sort_index()
    
    # Convert index to datetime for better plotting (use first day of each week)
    weekly_counts.index = weekly_counts.index.to_timestamp()
    
    # Plot
    plt.figure(figsize=(14, 6))
    plt.plot(weekly_counts.index, weekly_counts.values, marker='o', linewidth=1.5, markersize=4)
    plt.xlabel('Week', fontsize=12)
    plt.ylabel('Number of Observations', fontsize=12)
    plt.title('Weekly AIS Observations Over Time', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"Total weeks: {len(weekly_counts)}")
    print(f"Total observations: {weekly_counts.sum():,}")
    print(f"Average observations per week: {weekly_counts.mean():.0f}")
    print(f"Min observations in a week: {weekly_counts.min():,}")
    print(f"Max observations in a week: {weekly_counts.max():,}")
    
    return weekly_counts

# Use the function
daily_obs = plot_daily_observations(df)

In [ ]:
df

# Loading models and data

## Loading run 

In [ ]:


client = MlflowClient()            # or MlflowClient(tracking_uri)
experiment_id = "964646345580301318"

exp = MlflowClient().get_experiment(experiment_id) 
runs = MlflowClient().search_runs(experiment_id)


In [ ]:
runs